In [2]:
#GWAS file
import pandas as pd
phylogeny_file = pd.read_csv("phylogeny.csv")
yield_file = pd.read_csv("yield.csv")
GWAS_file = pd.merge(phylogeny_file,yield_file, on = "cultivar id", how = "left")
GWAS_file.to_csv("GWAS_file.csv",index=False)

In [3]:
from sklearn.linear_model import LogisticRegression

lr_height = LogisticRegression(max_iter=1000, random_state=42)


In [7]:
import pandas as pd
import numpy as np
data = pd.read_csv('GWAS_file.csv')
dicc = {'A': 0, 'T': 1, 'C': 2, 'G': 3}
encoded_sequences = []
for sequence in data['sequences']:
    encoded_sequence = np.array([dicc[nucleotide] for nucleotide in sequence])
    encoded_sequences.append(encoded_sequence)
encoded_data = pd.DataFrame(encoded_sequences)
encoded_data = pd.concat([data, encoded_data], axis=1)
encoded_data.to_csv("yencoded.csv")
print(encoded_data)


     Cluster ID cultivar id subpopulation  sequences  mean_height  Yield  0   
0             1        C001        Indica  GCTTTTCCC       138.27  30.69  3  \
1             1        W261  Intermediate  GCTTTTCCC       138.27  30.69  3   
2             2        W306   VI/Aromatic  GCTGTTCCC       129.60  28.41  3   
3             3        C128        Indica  GCTTTTTCC       156.07  28.04  3   
4             3        C180        Indica  GCTTTTTCC       156.07  28.04  3   
..          ...         ...           ...        ...          ...    ... ..   
523          73        W071        Indica  GCCGTATTT       117.34  37.47  3   
524          73        W146        Indica  GCCGTATTT       117.34  37.47  3   
525          73        W178        Indica  GCCGTATTT       117.34  37.47  3   
526          73        W221        Indica  GCCGTATTT       117.34  37.47  3   
527          73        W310        Indica  GCCGTATTT       117.34  37.47  3   

     1  2  3  4  5  6  7  8  
0    2  1  1  1  1  2

In [60]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
encoded_data = pd.read_csv('yencoded.csv')
X = encoded_data.iloc[:, 7:].values
y_yield = encoded_data['Yield'].values
X_train, X_test, y_yield_train, y_yield_test = train_test_split(X, y_yield, test_size=0.2, random_state=42)
rf_regressor = RandomForestRegressor(n_estimators=120, random_state=47)
rf_regressor.fit(X_train, y_yield_train)
new_seq = input("Enter DNA sequence: ")
print(new_seq)
new_seq_encoded = np.array([dicc[nucleotide] for nucleotide in new_seq])
new_seq_encoded = new_seq_encoded.reshape(1, -1)
predicted_yield = rf_regressor.predict(new_seq_encoded)
print("Predicted Yield:", predicted_yield)


GCTTTTTTT
Predicted Yield: [29.15]


In [64]:
from sklearn.metrics import mean_squared_error, r2_score

y_yield_pred = rf_regressor.predict(X_test)
mse = mean_squared_error(y_yield_test, y_yield_pred)
r_squared = r2_score(y_yield_test, y_yield_pred)
print("Mean Squared Error:", mse)
print("R-squared:", str(round(r_squared*100,2))+"%")


Mean Squared Error: 11.685606810010533
R-squared: 80.2%


In [65]:
import pickle
with open('model.pkl', 'wb') as file:
    pickle.dump(rf_regressor, file)
